## Exploratory Data Analysis

In this notebook the data given by different excel files is explored in order to define the details and constrains in the desing of the Data Model.

In [2]:
# Importar librerias python
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Importar modulos propios en notebook
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

# Verifica que el directorio fue agregado
print("Directorio en sys.path:", project_root in sys.path)

In [ ]:
from db.connection import engine_setting, engine_connection

In [ ]:


rootFolder = Path.cwd().parent

excel_path_1_bonos_trab =  rootFolder / "data" / "U532-2024-11-05.xls"
excel_path_2_taller_orden_rep =  rootFolder / "data" / "U533-2024-11-05.xls"
excel_path_3_bonos_pres =  rootFolder / "data" / "U551-2024-11-05.xls"
excel_path_4_stock =  rootFolder / "data" / "U552-2024-11-05.xls"
excel_path_5_compras =  rootFolder / "data" / "U553-2024-11-05.xls"
excel_path_6_mostrador =  rootFolder / "data" / "U554-2024-11-05.xls"
excel_path_7_invertidas =  rootFolder / "data" / "U555-2024-11-05.xls"
excel_path_8_taller_orden_venta =  rootFolder / "data" / "U560-2024-11-05.xls"
excel_path_9_cliente =  rootFolder / "data" / "U6301303-conta-2024-11-06.xls"
excel_path_10_articulo =  rootFolder / "data" / "U6301303-taller-2024-11-05.xls"
excel_path_11_empresas =  rootFolder / "data" / "U6311303-conta-2024-11-06.xls"
excel_path_12_talleres =  rootFolder / "data" / "U6311303-taller-2024-11-05.xls"
excel_path_13_almacenes =  rootFolder / "data" / "U6321303-2024-11-05.xls"
excel_path_14_operarios =  rootFolder / "data" / "U6331303-2024-11-05.xls"
excel_path_15_vehiculo =  rootFolder / "data" / "U6341303-2024-11-05.xls"
excel_path_16_tipos_horas =  rootFolder / "data" / "ULSTTHPT-2024-11-05.xls"
excel_path_17_tipos_venta =  rootFolder / "data" / "ULSTTVPT-2024-11-05.xls"

In [ ]:
# Leer el archivo Excel con la ruta completa
df_bonos_trab = pd.read_excel(excel_path_1_bonos_trab)
df_taller_orden_rep = pd.read_excel(excel_path_2_taller_orden_rep)
df_bono_pres = pd.read_excel(excel_path_3_bonos_pres)
df_stock = pd.read_excel(excel_path_4_stock)
df_compras = pd.read_excel(excel_path_5_compras)
df_mostrador = pd.read_excel(excel_path_6_mostrador)
df_invertidas = pd.read_excel(excel_path_7_invertidas)
df_taller_orden_venta = pd.read_excel(excel_path_8_taller_orden_venta)
df_cliente = pd.read_excel(excel_path_9_cliente)
df_articulos = pd.read_excel(excel_path_10_articulo)
df_empresas = pd.read_excel(excel_path_11_empresas)
df_talleres = pd.read_excel(excel_path_12_talleres)
df_almacenes = pd.read_excel(excel_path_13_almacenes)
df_operarios = pd.read_excel(excel_path_14_operarios)
df_vehiculos = pd.read_excel(excel_path_15_vehiculo)
df_tipos_horas = pd.read_excel(excel_path_16_tipos_horas)
df_tipos_venta = pd.read_excel(excel_path_17_tipos_venta)



### Conexión con capa bronze

In [ ]:
# = Connection to schema ddbb =

# Define database type and port 
db_type = 'postgresql' 
port = 5432

# Get schema name acording to layer name
db_schema = 'bronze'

# Create engine for the specific schema
engine = engine_setting(db_type=db_type, db_port=port, db_schema=db_schema)

# Establish the connection
conn = engine_connection(engine)

### Funciones de limpieza y transformación de datos.

- Eliminar columnas completamente vacias.
- Eliminar espacios vacios en cada registro.
- Verificar las columns que podrían ser números enteros 11 en lugar de 11.0. 

In [ ]:
def clean_df (df):
    '''
    '''
    df.dropna(how='all', inplace=True)

    for columun in df.columns:
        df = df[~df[columun].str.contains('\*\*\*', na=False)]
    
    for col in df.columns:
        
        if np.issubdtype(df[col].dtype, np.floating):
           
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype('int')
            df[col] = df[col].replace(0, None)


In [ ]:
def column_to_str (df):
    '''
    Cleans each column in the dataframe by removing leading and trailing whitespace 
    and ensures that all values are of string type.
    
    input:
        df (pd.DataFrame): DataFrame with columns to be cleaned and converted.
    
    return:
        df (pd.DataFrame): DataFrame with cleaned columns and values converted to string type.
    '''
    for column in df.columns:
        df[column] = df[column].str.lstrip()
        df[column] = df[column].str.rstrip()
        df[column] = df[column].astype(str)

## Data upload - Master tables

### Update new data

**Step 1**: Get existing values of the reference column in ddbb table.

**Step 2**: Compare dataframe values with ddbb existing values.

**Step 3**: If there are new values, values are going to be written in th bbdd.

**Considerations**
- All handle values need to be str type.
- Clean white spaces for all values.


In [ ]:
def get_existing_values (table_name, column_name):
    '''
    Get existing values of key column from table in ddbb.

    input:
        table_name (str): table name in database.
        column_name (sts): column name of table in database.

    output:
        existing_values (list): list of exisitng values in database.  
    '''
    query = f'SELECT "{column_name}" FROM "{table_name}"'
    existing_values = pd.read_sql(query, con=engine)[column_name].tolist()
    
    return(existing_values)

In [ ]:
def get_new_values (df_name, key_column_name, existing_values):
    '''
    Get the dataframe rows which not exist in table database.

    input: 
        df_name (str): dataframe that is compared with existing values in ddbb.
        key_column_name (str): key column name of the dataframe to be compared.
        existing_values (list): list of existing values in ddbb. 
    '''
    df_new_values = df_name[~df_name[key_column_name].isin(existing_values)]
    
    return df_new_values

In [ ]:
def write_new_values (df_new_values, db_table_name):
    '''
    Write new found values in ddbb table. 
    '''
    df_new_values.to_sql(db_table_name, con=engine, if_exists='append', index=False)



In [ ]:
# One function 

def upload_new_data (table_name, df, key_column, engine):
    '''
    Process new values: 
    - Get existing values from the database.
    - Identify new values in the dataframe.
    - Write new values to the database table.

    input: 
        table_name (str): name of the table in the database.
        df (DataFrame): dataframe to be compared with existing values.
        key_column (str): key column name in the dataframe and table database to compare.
        engine (SQLAlchemy engine): database connection.

    output:
        None
    '''
    # Get existing values from the database
    query = f'SELECT "{key_column}" FROM "{table_name}"'
    existing_values = pd.read_sql(query, con=engine)[key_column].tolist()

    # Get new data by comparing dataframe with existing values
    df_new_data = df[~df[key_column].isin(existing_values)]

    # Write new values to the database
    if not df_new_data.empty:
        df_new_data.to_sql(table_name, con=engine, if_exists='append', index=False)


### Update Modified Data

In [ ]:
# Load data form database

def get_database_table (table_name):

    query = f'SELECT * FROM "{table_name}"'
    db_table = pd.read_sql(query, con=engine)

    return (db_table)

In [ ]:
def get_modified_data (df, db_table, key_column):
    '''
    Verify and get modified data comparing to existing data in database.

    input:
        - df (DataFrame): DataFrame to compare.
        - db_table (DataFrame): Dataframe with existing data from database.
        - key_colum (str): name of key column with is going to be used to merge both DataFrames.
    
    output:
        - modified_data (DataFrame): DataFrame with modified data. 
    '''
    merge_data = df.merge(
        db_table, on=key_column,
        suffixes=('_new', '_db')
    )

    # Make sure both DataFrames have the same column names and index
    new_merge_data = merge_data.filter(regex='_new$').sort_index(axis=1)
    db_data = merge_data.filter(regex='_db$').sort_index(axis=1)

    # Otherwise column names are adjusted
    new_merge_data.columns = new_merge_data.columns.str.replace('_new$', '', regex=True)
    db_data.columns = db_data.columns.str.replace('_db$', '', regex=True)

    # Compare rows where there are differences
    modified_data = merge_data[(new_merge_data != db_data).any(axis=1)]

    return modified_data
    

In [ ]:
def update_modified_data (modified_data, key_column, db_table_name):
    '''
    description
    '''
    for _, fila in modified_data.iterrows():
        # Create dictionary of columns with nuew values
        valores = {col.replace('_new', ''): fila[col] for col in modified_data.filter(regex='_new$').columns}
        key_value = fila[key_column]

        # Excecute update query
        set_clause = ', '.join([f'"{col}" = :{col}' for col in valores.keys()])
        query = f'UPDATE "{db_table_name}" SET {set_clause} WHERE "{key_column}" = :key_value'
        valores['key_value'] = key_value
        engine.execute(query, valores)
        print(f"{len(modified_data)} registros actualizados en '{db_table_name}'.")


Tabla Empresas

In [ ]:
# Simulación actualización de nuevos datos

# Datos a verificar
df_empresas

# Convertir cada columna individualmente a texto
for col in df_empresas.columns:
    df_empresas[col] = df_empresas[col].astype(str)

df_empresas.info()

In [ ]:
# Datos existentes en base de datos
db_empresas = df_empresas.head(1).copy() 

# Convertir cada columna individualmente a texto
for col in db_empresas.columns:
    db_empresas[col] = db_empresas[col].astype(str)

db_empresas.info()

In [ ]:
db_empresas

In [ ]:
checking_values = df_empresas['Codig']
print(checking_values)

In [ ]:
# Comparación de valores 
existing_values = db_empresas['Codig'].tolist()


In [ ]:
print(existing_values)
print(df_empresas['Codig'])

In [ ]:
df_empresas_new = df_empresas[~df_empresas['Codig'].isin(existing_values)]
df_empresas_new

In [ ]:
# Upload new data execution
upload_new_data ('Empresas U6311303', df_empresas, 'Codig', engine)

Update modified data

In [ ]:
# simulación 

df_empresas_mod = df_empresas.copy()
df_empresas_mod.loc[df_empresas_mod['Codig'] == '13', 'Nombre'] = 'Nombre Empresa Modificado'
df_empresas_mod

In [ ]:
df_empresas

In [ ]:
merge_data = df_empresas_mod.merge(
        df_empresas, on='Codig',
        suffixes=('_new', '_db')
    )
merge_data

In [ ]:
# Filter and make sure both DataFrames have the same column names and index
new_merge_data = merge_data.filter(regex='_new$').sort_index(axis=1)
db_data = merge_data.filter(regex='_db$').sort_index(axis=1)

# Otherwise column names are adjusted
new_merge_data.columns = new_merge_data.columns.str.replace('_new$', '', regex=True)
db_data.columns = db_data.columns.str.replace('_db$', '', regex=True)

# Compare rows where there are differences
modified_data = merge_data[(new_merge_data != db_data).any(axis=1)]

modified_data

Tabla Almacenes

In [ ]:
df_almacenes.head()

Tabla Operarios

In [ ]:
df_operarios.head()

In [ ]:
df_operarios.tail()

In [ ]:
df_operarios.dropna(how='all', inplace=True)

In [ ]:
df_operarios.tail()

In [ ]:
df_operarios = df_operarios[~df_operarios['Codigo'].str.contains('\*\*\*', na=False)]

In [ ]:
df_operarios.shape

Tabla vehículos

In [ ]:
df_vehiculos.head()

In [ ]:
for col in df_vehiculos.columns:
    if np.issubdtype(df_vehiculos[col].dtype, np.floating):  # Verificamos si la columna es de tipo float
        # Reemplazar NaN por 0
        df_vehiculos[col] = df_vehiculos[col].fillna(0)
        
        # Convertir la columna a tipo entero
        df_vehiculos[col] = df_vehiculos[col].astype('int')
        
        # Reemplazar 0 por NaN
        df_vehiculos[col] = df_vehiculos[col].replace(0, None)

In [ ]:
df_vehiculos.head()

In [ ]:
df_vehiculos.info()

Tabla Tipos Horas

In [ ]:
df_tipos_horas.head()

In [ ]:
df_tipos_horas

Tabla Tipos Ventas

In [ ]:
df_tipos_venta.head()

In [ ]:
df_tipos_venta

Tabla Cliente

In [ ]:
df_cliente.head()

In [ ]:
df_cliente

Tabla Articulos

In [ ]:
df_articulos.head()

In [ ]:
df_articulos

In [ ]:
df_talleres.head()

Tabla Talleres

In [ ]:
df_talleres

## Data update - Fact tables

In [ ]:
df_bonos_trab

In [ ]:
df_taller_orden_rep

In [ ]:
df_bono_pres

In [ ]:
df_stock

In [ ]:
df_compras

In [ ]:
df_mostrador

In [ ]:
df_invertidas

In [ ]:
df_taller_orden_venta

In [ ]:
# maestros
# df_empresas.to_sql('Empresas U6311303', con=engine, if_exists='append', index=False) # ok 
# df_almacenes.to_sql('Almacenes U6321303', con=engine, if_exists='append', index=False) # ok 
# df_operarios.to_sql('Operarios U6331303', con=engine, if_exists='append', index=False) # ok
# df_vehiculos.to_sql('Vehiculo U6341303', con=engine, if_exists='append', index=False) # ok
# df_tipos_horas.to_sql('TipoHoras ULSTTHPT', con=engine, if_exists='append', index=False) # ok
# df_tipos_venta.to_sql('TipoVentasAlmacen ULSTTVPT', con=engine, if_exists='append', index=False) # ok
# df_cliente.to_sql('Cliente U6301303', con=engine, if_exists='append', index=False) # ok 
# df_articulos.to_sql('Articulo U6301303', con=engine, if_exists='append', index=False) # ok
# df_talleres.to_sql('Talleres U6311303', con=engine, if_exists='append', index=False) # ok 

# # hechos
# df_bonos_trab.to_sql('BonosTrabajadas U532', con=engine, if_exists='append', index=False) # ok 
# df_taller_orden_rep.to_sql('Taller OrdenReparacion U533', con=engine, if_exists='append', index=False) # ok
# df_bono_pres.to_sql('BonosPresencia U551', con=engine, if_exists='append', index=False) # ok
# df_stock.to_sql('Stock U552', con=engine, if_exists='append', index=False) # ok
# df_compras.to_sql('Compras U553', con=engine, if_exists='append', index=False) # ok
# df_mostrador.to_sql('Taller OrdenVentaMostrador U554', con=engine, if_exists='append', index=False) # ok
# df_invertidas.to_sql('Invertidas U555', con=engine, if_exists='append', index=False) # ok
# df_taller_orden_venta.to_sql('Taller OrdenVentaTaller U560', con=engine, if_exists='append', index=False) # ok


In [ ]:
if conn:
    conn.close()
